# Testing Chat GPT 3.5's ability to write basic simpy code.

The initial questioning is based on a adapted simplified version of 

> Monks, T., Worthington, D., Allen, M. et al. A modelling tool for capacity planning in acute and community stroke services. BMC Health Serv Res 16, 530 (2016). https://doi.org/10.1186/s12913-016-1789-4

One key difference is that the model in the paper is an infinite queuing model that straddles both acute and rehab.  In this example, we allow queuing and only model the acute length of stay.  There are three classes of patient arriving to the system.


## Test 1

1. Can ChatGPT produce simpy code that runs? 
2. Is it logical and are multiple arrival classes implemented correctly?

> Quick limitations.  I didn't specify python or simpy version or any other libraries.  Didn't specify results collection procudure.  Didn't specify anything about controlling random sampling.

### Environment used for test

```yaml
name: gen_simpy
channels:
  - conda-forge
dependencies:
  - joblib=1.2.0
  - jupyterlab=4.0.8
  - jupyterlab-plotly-extension=1.0.0
  - jupyterlab-spellchecker=0.8.4
  - matplotlib=3.8.1
  - numpy=1.26.0
  - pandas=2.1.3
  - pip=23.3.1
  - python=3.10.13
  - simpy=4.1.1
```

### Input

```
write a simulation model in python using the package simpy.  The model should simulate an acute stroke unit (ASU). The stroke unit manages three types of patient: strokes, transient ischemic strokes (TIA), and patients with complex neurological problems.  

The  ASU currently has 9 inpatient beds available. When a patient is admitted they undergo a period
of treatment and then are discharged elsewhere.  Each type of patient has its own arrival distribution and own length of stay distribution.

The inter-arrival time of stroke patients to the model is exponentially distributed with mean 1.2 days
The inter-arrival time of TIA patients to the model is exponentially distributed with mean 9.5 days
The inter-arrival time of complex neurological patients to the model is exponentially distributed with mean 3.5 days

The length of stay for stroke patients is log normally distributed with a mean 7.4 days and standard deviation 8.5 days

The length of stay for TIA patients is log normally distributed with a mean 1.8 days and standard deviation 2.3 days

The length of stay for TIA patients is log normally distributed with a mean 2.0 days and standard deviation 2.5 days
```

## Output


In [1]:
import simpy
import numpy as np
import random

class AcuteStrokeUnit:
    def __init__(self, env, num_beds):
        self.env = env
        self.beds = simpy.Resource(env, capacity=num_beds)

    def admit_patient(self, patient_type):
        return self.beds.request()

    def discharge_patient(self, bed):
        return self.beds.release(bed)

def stroke_patient_arrival(env, asu):
    while True:
        inter_arrival_time = np.random.exponential(scale=1.2)
        yield env.timeout(inter_arrival_time)
        env.process(admit_and_discharge(env, asu, "Stroke"))

def tia_patient_arrival(env, asu):
    while True:
        inter_arrival_time = np.random.exponential(scale=9.5)
        yield env.timeout(inter_arrival_time)
        env.process(admit_and_discharge(env, asu, "TIA"))

def complex_neuro_patient_arrival(env, asu):
    while True:
        inter_arrival_time = np.random.exponential(scale=3.5)
        yield env.timeout(inter_arrival_time)
        env.process(admit_and_discharge(env, asu, "Complex Neuro"))

def admit_and_discharge(env, asu, patient_type):
    bed_request = yield asu.admit_patient(patient_type)
    length_of_stay = get_length_of_stay(patient_type)
    yield env.timeout(length_of_stay)
    asu.discharge_patient(bed_request)

def get_length_of_stay(patient_type):
    if patient_type == "Stroke":
        return np.random.lognormal(mean=np.log(7.4), sigma=np.log(8.5))
    elif patient_type == "TIA":
        return np.random.lognormal(mean=np.log(1.8), sigma=np.log(2.3))
    elif patient_type == "Complex Neuro":
        return np.random.lognormal(mean=np.log(2.0), sigma=np.log(2.5))

# Simulation setup
env = simpy.Environment()
asu = AcuteStrokeUnit(env, num_beds=9)

# Start patient arrival processes
env.process(stroke_patient_arrival(env, asu))
env.process(tia_patient_arrival(env, asu))
env.process(complex_neuro_patient_arrival(env, asu))

# Run the simulation
env.run(until=100)  # Adjust the simulation time as needed

### Observations on output

* There's is no code to show the model running e.g. patients arriving etc.
* `AcuteStrokeUnit.admit_patient` accepts a parameter called `patient_type`. This is not used.
* imports python's `random` module but does not use it (uses numpy instead - this is what I would do)
* Sets up independent generator functions for each arrival type.  This is what I would do.
* On inspection the `get_length_of_stay` function appears to be correct.  Quite a neat implemention as well!
* I didn't specify the simulation time in the input to ChatGPT and it provided an "adjust as needed" comment.


## Test 2

### Input

In this test I provided ChatGPT the identical input to test 1.  It generated slightly different code.

### Output

In [3]:
import simpy
import numpy as np
import scipy.stats as stats

class AcuteStrokeUnit:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=9)

    def stroke_patient(self, patient_id):
        arrival_time = self.env.now
        LOS = self.generate_length_of_stay(mean=7.4, std_dev=8.5)

        with self.beds.request() as bed_request:
            yield bed_request
            yield self.env.timeout(LOS)

        print(f"Stroke Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def tia_patient(self, patient_id):
        arrival_time = self.env.now
        LOS = self.generate_length_of_stay(mean=1.8, std_dev=2.3)

        with self.beds.request() as bed_request:
            yield bed_request
            yield self.env.timeout(LOS)

        print(f"TIA Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def complex_neuro_patient(self, patient_id):
        arrival_time = self.env.now
        LOS = self.generate_length_of_stay(mean=2.0, std_dev=2.5)

        with self.beds.request() as bed_request:
            yield bed_request
            yield self.env.timeout(LOS)

        print(f"Complex Neuro Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def generate_length_of_stay(self, mean, std_dev):
        return max(0, np.random.lognormal(mean=np.log(mean**2 / np.sqrt(std_dev**2 + mean**2)),
                                           sigma=np.sqrt(np.log(1 + (std_dev**2 / mean**2)))))

def patient_generator(env, asu):
    patient_id = 1

    while True:
        inter_arrival_stroke = np.random.exponential(scale=1.2)
        inter_arrival_tia = np.random.exponential(scale=9.5)
        inter_arrival_neuro = np.random.exponential(scale=3.5)

        yield env.timeout(inter_arrival_stroke)
        env.process(asu.stroke_patient(patient_id))
        patient_id += 1

        yield env.timeout(inter_arrival_tia)
        env.process(asu.tia_patient(patient_id))
        patient_id += 1

        yield env.timeout(inter_arrival_neuro)
        env.process(asu.complex_neuro_patient(patient_id))
        patient_id += 1

env = simpy.Environment()
asu = AcuteStrokeUnit(env)
env.process(patient_generator(env, asu))
env.run(until=100)  # Adjust the simulation time as needed

TIA Patient 2 admitted at 5.44, discharged at 6.68
Stroke Patient 1 admitted at 0.98, discharged at 11.91
Complex Neuro Patient 3 admitted at 12.36, discharged at 13.04
Stroke Patient 4 admitted at 14.19, discharged at 17.04
TIA Patient 5 admitted at 20.85, discharged at 22.91
Complex Neuro Patient 6 admitted at 21.55, discharged at 23.16
TIA Patient 8 admitted at 26.22, discharged at 26.53
Stroke Patient 7 admitted at 22.98, discharged at 27.48
Complex Neuro Patient 9 admitted at 26.55, discharged at 28.22
TIA Patient 11 admitted at 30.88, discharged at 31.45
Stroke Patient 10 admitted at 26.75, discharged at 35.39
Complex Neuro Patient 12 admitted at 38.16, discharged at 39.48
Stroke Patient 13 admitted at 38.48, discharged at 46.00
Complex Neuro Patient 15 admitted at 46.32, discharged at 46.66
TIA Patient 14 admitted at 42.70, discharged at 47.45
TIA Patient 17 admitted at 63.74, discharged at 64.01
Complex Neuro Patient 18 admitted at 65.77, discharged at 65.89
Stroke Patient 19 a

### Observations on output

* I had to install scipy as I had not included it in the original environment.  does not appear to be used anyway. Perhaps we can ask ChatGPT to produce the environment as well as specifiying it?
* arrival generator appears to be incorrect.  It forces inter-arrivals order of stroke-> tia->complex neuro.  This means that IAT of each class is incorrect and different from expected.
* LoS process seems correct for each patient type.  Seems to convert mean stdev to params for lognormal - although not checked formulas.

## Test 2a (modification 1)

Given that the arrival of patients to the system is not handled correct in the above code I modified the input as follows.

### Input (Modification)

```
modify the code so that patient arrivals for stroke, tia and complex neurological are generated by separate functions.
```

### Output

In [4]:
import simpy
import numpy as np
import scipy.stats as stats

class AcuteStrokeUnit:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=9)

    def stroke_patient(self, patient_id):
        arrival_time = self.env.now
        LOS = self.generate_length_of_stay(mean=7.4, std_dev=8.5)

        with self.beds.request() as bed_request:
            yield bed_request
            yield self.env.timeout(LOS)

        print(f"Stroke Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def tia_patient(self, patient_id):
        arrival_time = self.env.now
        LOS = self.generate_length_of_stay(mean=1.8, std_dev=2.3)

        with self.beds.request() as bed_request:
            yield bed_request
            yield self.env.timeout(LOS)

        print(f"TIA Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def complex_neuro_patient(self, patient_id):
        arrival_time = self.env.now
        LOS = self.generate_length_of_stay(mean=2.0, std_dev=2.5)

        with self.beds.request() as bed_request:
            yield bed_request
            yield self.env.timeout(LOS)

        print(f"Complex Neuro Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def generate_length_of_stay(self, mean, std_dev):
        return max(0, np.random.lognormal(mean=np.log(mean**2 / np.sqrt(std_dev**2 + mean**2)),
                                           sigma=np.sqrt(np.log(1 + (std_dev**2 / mean**2)))))

def stroke_patient_generator(env, asu):
    patient_id = 1
    while True:
        inter_arrival = np.random.exponential(scale=1.2)
        yield env.timeout(inter_arrival)
        env.process(asu.stroke_patient(patient_id))
        patient_id += 1

def tia_patient_generator(env, asu):
    patient_id = 1
    while True:
        inter_arrival = np.random.exponential(scale=9.5)
        yield env.timeout(inter_arrival)
        env.process(asu.tia_patient(patient_id))
        patient_id += 1

def complex_neuro_patient_generator(env, asu):
    patient_id = 1
    while True:
        inter_arrival = np.random.exponential(scale=3.5)
        yield env.timeout(inter_arrival)
        env.process(asu.complex_neuro_patient(patient_id))
        patient_id += 1

env = simpy.Environment()
asu = AcuteStrokeUnit(env)

env.process(stroke_patient_generator(env, asu))
env.process(tia_patient_generator(env, asu))
env.process(complex_neuro_patient_generator(env, asu))

env.run(until=30)  # Adjust the simulation time as needed


Stroke Patient 2 admitted at 2.70, discharged at 6.13
Stroke Patient 1 admitted at 1.22, discharged at 6.36
Stroke Patient 5 admitted at 5.60, discharged at 6.97
Stroke Patient 3 admitted at 3.57, discharged at 8.30
Stroke Patient 8 admitted at 7.81, discharged at 10.54
TIA Patient 1 admitted at 11.11, discharged at 11.47
Stroke Patient 4 admitted at 4.74, discharged at 16.96
Stroke Patient 6 admitted at 6.13, discharged at 18.08
Stroke Patient 7 admitted at 7.46, discharged at 18.24
Stroke Patient 9 admitted at 11.40, discharged at 19.04
Stroke Patient 12 admitted at 16.93, discharged at 19.30
Complex Neuro Patient 1 admitted at 17.60, discharged at 19.80
Stroke Patient 13 admitted at 18.01, discharged at 22.35
Complex Neuro Patient 2 admitted at 21.49, discharged at 22.41
Stroke Patient 15 admitted at 19.95, discharged at 23.91
Stroke Patient 14 admitted at 19.83, discharged at 24.16
Stroke Patient 16 admitted at 20.58, discharged at 24.22
Stroke Patient 19 admitted at 21.57, dischar

### Observations on the code

* The generation of patients is now correctly handled.  But there is an issue with the patient number. Given that functions are being used one way to handle this is with a module level variable or a string such as "stroke1", "stroke2", "tia1".

## Test 2b (modification)

```
modify the code to ensure that each stroke, tia and complex neurological patient has a globally unique identifier.
```

In [5]:
import simpy
import numpy as np

class AcuteStrokeUnit:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=9)
        self.stroke_counter = 1
        self.tia_counter = 1
        self.neuro_counter = 1

    def stroke_patient(self):
        patient_id = self.stroke_counter
        self.stroke_counter += 1

        arrival_time = self.env.now
        LOS = self.generate_length_of_stay(mean=7.4, std_dev=8.5)

        with self.beds.request() as bed_request:
            yield bed_request
            yield self.env.timeout(LOS)

        print(f"Stroke Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def tia_patient(self):
        patient_id = self.tia_counter
        self.tia_counter += 1

        arrival_time = self.env.now
        LOS = self.generate_length_of_stay(mean=1.8, std_dev=2.3)

        with self.beds.request() as bed_request:
            yield bed_request
            yield self.env.timeout(LOS)

        print(f"TIA Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def complex_neuro_patient(self):
        patient_id = self.neuro_counter
        self.neuro_counter += 1

        arrival_time = self.env.now
        LOS = self.generate_length_of_stay(mean=2.0, std_dev=2.5)

        with self.beds.request() as bed_request:
            yield bed_request
            yield self.env.timeout(LOS)

        print(f"Complex Neuro Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def generate_length_of_stay(self, mean, std_dev):
        return max(0, np.random.lognormal(mean=np.log(mean**2 / np.sqrt(std_dev**2 + mean**2)),
                                           sigma=np.sqrt(np.log(1 + (std_dev**2 / mean**2)))))

def stroke_patient_generator(env, asu):
    while True:
        inter_arrival = np.random.exponential(scale=1.2)
        yield env.timeout(inter_arrival)
        env.process(asu.stroke_patient())

def tia_patient_generator(env, asu):
    while True:
        inter_arrival = np.random.exponential(scale=9.5)
        yield env.timeout(inter_arrival)
        env.process(asu.tia_patient())

def complex_neuro_patient_generator(env, asu):
    while True:
        inter_arrival = np.random.exponential(scale=3.5)
        yield env.timeout(inter_arrival)
        env.process(asu.complex_neuro_patient())

env = simpy.Environment()
asu = AcuteStrokeUnit(env)

env.process(stroke_patient_generator(env, asu))
env.process(tia_patient_generator(env, asu))
env.process(complex_neuro_patient_generator(env, asu))

env.run(until=30)  # Adjust the simulation time as needed


Complex Neuro Patient 1 admitted at 0.22, discharged at 0.50
Stroke Patient 1 admitted at 0.11, discharged at 0.77
Stroke Patient 2 admitted at 0.83, discharged at 4.31
Stroke Patient 3 admitted at 1.21, discharged at 5.02
Stroke Patient 4 admitted at 2.05, discharged at 5.15
Complex Neuro Patient 2 admitted at 5.38, discharged at 5.84
Stroke Patient 5 admitted at 4.32, discharged at 8.47
Stroke Patient 7 admitted at 7.91, discharged at 9.48
TIA Patient 1 admitted at 10.13, discharged at 10.34
Stroke Patient 8 admitted at 10.45, discharged at 13.24
TIA Patient 2 admitted at 12.57, discharged at 13.57
TIA Patient 3 admitted at 13.80, discharged at 14.34
TIA Patient 4 admitted at 15.61, discharged at 16.40
Complex Neuro Patient 3 admitted at 16.26, discharged at 16.88
Complex Neuro Patient 4 admitted at 16.51, discharged at 17.03
Complex Neuro Patient 5 admitted at 16.65, discharged at 17.55
Stroke Patient 12 admitted at 15.23, discharged at 18.63
Stroke Patient 13 admitted at 15.30, dis

## Observations

* This is sensible and allows for unique ids.

## Test 3: modification to include data collection.

```
modify the code: at the end of the simulation calculate the mean waiting time for a bed and the mean utilization of the beds.
```

In [8]:
import simpy
import numpy as np

class AcuteStrokeUnit:
    def __init__(self, env):
        self.env = env
        self.beds = simpy.Resource(env, capacity=9)
        self.stroke_counter = 1
        self.tia_counter = 1
        self.neuro_counter = 1
        self.waiting_times = []  # Track waiting times for each patient
        self.utilization_times = []  # Track time beds are utilized

    def stroke_patient(self):
        patient_id = self.stroke_counter
        self.stroke_counter += 1

        arrival_time = self.env.now
        with self.beds.request() as bed_request:
            yield bed_request
            wait_time = self.env.now - arrival_time
            self.waiting_times.append(wait_time)

            LOS = self.generate_length_of_stay(mean=7.4, std_dev=8.5)
            yield self.env.timeout(LOS)

        print(f"Stroke Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def tia_patient(self):
        patient_id = self.tia_counter
        self.tia_counter += 1

        arrival_time = self.env.now
        with self.beds.request() as bed_request:
            yield bed_request
            wait_time = self.env.now - arrival_time
            self.waiting_times.append(wait_time)

            LOS = self.generate_length_of_stay(mean=1.8, std_dev=2.3)
            yield self.env.timeout(LOS)

        print(f"TIA Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def complex_neuro_patient(self):
        patient_id = self.neuro_counter
        self.neuro_counter += 1

        arrival_time = self.env.now
        with self.beds.request() as bed_request:
            yield bed_request
            wait_time = self.env.now - arrival_time
            self.waiting_times.append(wait_time)

            LOS = self.generate_length_of_stay(mean=2.0, std_dev=2.5)
            yield self.env.timeout(LOS)

        print(f"Complex Neuro Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

        # Track utilization time at the end of each patient's stay
        utilization_time = self.env.now - arrival_time
        self.utilization_times.append(utilization_time)

    def generate_length_of_stay(self, mean, std_dev):
        return max(0, np.random.lognormal(mean=np.log(mean**2 / np.sqrt(std_dev**2 + mean**2)),
                                           sigma=np.sqrt(np.log(1 + (std_dev**2 / mean**2)))))

def stroke_patient_generator(env, asu):
    while True:
        inter_arrival = np.random.exponential(scale=1.2)
        yield env.timeout(inter_arrival)
        env.process(asu.stroke_patient())

def tia_patient_generator(env, asu):
    while True:
        inter_arrival = np.random.exponential(scale=9.5)
        yield env.timeout(inter_arrival)
        env.process(asu.tia_patient())

def complex_neuro_patient_generator(env, asu):
    while True:
        inter_arrival = np.random.exponential(scale=3.5)
        yield env.timeout(inter_arrival)
        env.process(asu.complex_neuro_patient())

env = simpy.Environment()
asu = AcuteStrokeUnit(env)

env.process(stroke_patient_generator(env, asu))
env.process(tia_patient_generator(env, asu))
env.process(complex_neuro_patient_generator(env, asu))

env.run(until=30)  # Adjust the simulation time as needed

# Calculate and print mean waiting time for a bed
mean_waiting_time = np.mean(asu.waiting_times)
print(f"Mean Waiting Time for a Bed: {mean_waiting_time:.2f} days")

# Calculate and print mean utilization of the beds
mean_utilization = np.mean(asu.utilization_times) / len(asu.beds.resourceList)
print(f"Mean Utilization of Beds: {mean_utilization:.2%}")


Complex Neuro Patient 1 admitted at 2.83, discharged at 4.47
Stroke Patient 2 admitted at 3.24, discharged at 6.89
Complex Neuro Patient 2 admitted at 5.89, discharged at 7.23
Stroke Patient 4 admitted at 6.22, discharged at 7.86
Complex Neuro Patient 3 admitted at 8.24, discharged at 8.62
Stroke Patient 1 admitted at 0.31, discharged at 8.83
Stroke Patient 3 admitted at 4.02, discharged at 9.92
Complex Neuro Patient 4 admitted at 8.95, discharged at 10.30
Complex Neuro Patient 5 admitted at 10.44, discharged at 10.66
TIA Patient 1 admitted at 12.34, discharged at 13.90
Complex Neuro Patient 6 admitted at 12.49, discharged at 14.10
Stroke Patient 8 admitted at 12.71, discharged at 17.43
TIA Patient 2 admitted at 16.57, discharged at 17.90
Stroke Patient 6 admitted at 10.68, discharged at 17.91
Complex Neuro Patient 7 admitted at 17.46, discharged at 18.30
Stroke Patient 7 admitted at 11.00, discharged at 19.00
Stroke Patient 9 admitted at 17.16, discharged at 20.29
TIA Patient 3 admitt

AttributeError: 'Resource' object has no attribute 'resourceList'

### Obserations

* I was unable to get ChatGPT to correctly calculate utilisation. Even with several explanations.  The problem was that it was unable to get the number of beds available.
* Instead I opted to get rid of hard coded parameters - this may also allow better results calcualation afterwards.

### Test 4

```
Modify the code so that all parameters are stored within a class called Experiment.  The list of parameters includes, 

1. the means of the arrival distributions
2. mean and stdev of the length of stay distributions
3. The number of beds.
```

In [23]:
import simpy
import numpy as np

class Experiment:
    def __init__(self):
        # Means of the arrival distributions
        self.mean_arrival_stroke = 1.2
        self.mean_arrival_tia = 9.5
        self.mean_arrival_neuro = 3.5

        # Mean and stdev of the length of stay distributions
        self.mean_los_stroke = 7.4
        self.std_dev_los_stroke = 8.5
        self.mean_los_tia = 1.8
        self.std_dev_los_tia = 2.3
        self.mean_los_neuro = 2.0
        self.std_dev_los_neuro = 2.5

        # Number of beds
        self.num_beds = 9
        self.simulation_time = 30

class AcuteStrokeUnit:
    def __init__(self, env, experiment):
        self.env = env
        self.beds = simpy.Resource(env, capacity=experiment.num_beds)
        self.stroke_counter = 1
        self.tia_counter = 1
        self.neuro_counter = 1
        self.waiting_times = []  # Track waiting times for each patient

    def stroke_patient(self):
        patient_id = self.stroke_counter
        self.stroke_counter += 1

        arrival_time = self.env.now
        with self.beds.request() as bed_request:
            yield bed_request
            wait_time = self.env.now - arrival_time
            self.waiting_times.append(wait_time)

            LOS = self.generate_length_of_stay(mean=experiment.mean_los_stroke, std_dev=experiment.std_dev_los_stroke)
            yield self.env.timeout(LOS)

        print(f"Stroke Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def tia_patient(self):
        patient_id = self.tia_counter
        self.tia_counter += 1

        arrival_time = self.env.now
        with self.beds.request() as bed_request:
            yield bed_request
            wait_time = self.env.now - arrival_time
            self.waiting_times.append(wait_time)

            LOS = self.generate_length_of_stay(mean=experiment.mean_los_tia, std_dev=experiment.std_dev_los_tia)
            yield self.env.timeout(LOS)

        print(f"TIA Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def complex_neuro_patient(self):
        patient_id = self.neuro_counter
        self.neuro_counter += 1

        arrival_time = self.env.now
        with self.beds.request() as bed_request:
            yield bed_request
            wait_time = self.env.now - arrival_time
            self.waiting_times.append(wait_time)

            LOS = self.generate_length_of_stay(mean=experiment.mean_los_neuro, std_dev=experiment.std_dev_los_neuro)
            yield self.env.timeout(LOS)

        print(f"Complex Neuro Patient {patient_id} admitted at {arrival_time:.2f}, discharged at {self.env.now:.2f}")

    def generate_length_of_stay(self, mean, std_dev):
        return max(0, np.random.lognormal(mean=np.log(mean**2 / np.sqrt(std_dev**2 + mean**2)),
                                           sigma=np.sqrt(np.log(1 + (std_dev**2 / mean**2)))))

def stroke_patient_generator(env, asu):
    while True:
        inter_arrival = np.random.exponential(scale=experiment.mean_arrival_stroke)
        yield env.timeout(inter_arrival)
        env.process(asu.stroke_patient())

def tia_patient_generator(env, asu):
    while True:
        inter_arrival = np.random.exponential(scale=experiment.mean_arrival_tia)
        yield env.timeout(inter_arrival)
        env.process(asu.tia_patient())

def complex_neuro_patient_generator(env, asu):
    while True:
        inter_arrival = np.random.exponential(scale=experiment.mean_arrival_neuro)
        yield env.timeout(inter_arrival)
        env.process(asu.complex_neuro_patient())

experiment = Experiment()
env = simpy.Environment()
asu = AcuteStrokeUnit(env, experiment)

env.process(stroke_patient_generator(env, asu))
env.process(tia_patient_generator(env, asu))
env.process(complex_neuro_patient_generator(env, asu))

env.run(until=experiment.simulation_time)  # Adjust the simulation time as needed


Stroke Patient 1 admitted at 2.27, discharged at 4.84
Stroke Patient 4 admitted at 6.35, discharged at 7.10
Complex Neuro Patient 1 admitted at 6.63, discharged at 7.48
Stroke Patient 2 admitted at 3.20, discharged at 8.85
Stroke Patient 6 admitted at 8.92, discharged at 10.41
Stroke Patient 7 admitted at 10.03, discharged at 11.41
Stroke Patient 5 admitted at 8.21, discharged at 11.52
Complex Neuro Patient 2 admitted at 7.05, discharged at 14.21
Complex Neuro Patient 3 admitted at 12.76, discharged at 14.41
Stroke Patient 10 admitted at 13.03, discharged at 14.74
Complex Neuro Patient 4 admitted at 13.10, discharged at 14.94
Stroke Patient 11 admitted at 13.17, discharged at 15.81
TIA Patient 1 admitted at 14.23, discharged at 15.93
Stroke Patient 12 admitted at 13.29, discharged at 16.74
Stroke Patient 9 admitted at 12.45, discharged at 17.53
Stroke Patient 3 admitted at 4.71, discharged at 18.77
Stroke Patient 8 admitted at 11.41, discharged at 18.87
Stroke Patient 13 admitted at 16